In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
directory = '/scratch/6256481/'


import sys
sys.path.insert(0, directory + f'python/py_code/')


##########################################
# import packages
##########################################
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely
import networkx as nx 
import itertools 
import os

##########################################
# partial modules
##########################################
from shapely.geometry import Point, LineString
from tqdm import tqdm
from glob import glob

##########################################
# function typecasting
##########################################
from typing import List, Tuple

##########################################
# import custom modules
##########################################
from support import get_local_utm_projection, find_connected_side
from line_functions import azimuth

##########################################
# impport reach DEF
##########################################
from reach_definition import *





# open File

In [15]:
vectorFile = np.sort(glob(directory +f'input/SWOT_vector/*reach*v17*.gpkg'))
nodeFile   = np.sort(glob(directory +f'input/SWOT_nodes/*node*v17*.gpkg'))
vectorFile[-2]

'/scratch/6256481/input/SWOT_vector/oc_sword_reaches_v17.gpkg'

In [3]:
dfIn     = gpd.read_file(vectorFile[-2])
dfNodeIn = gpd.read_file(nodeFile[-2])

ERROR 1: PROJ: proj_create_from_database: Open of /home/6256481/.conda/envs/RivAlg10/share/proj failed


# test reach def

In [4]:
min_reach_len_factor = 12*4

In [5]:
df, dfNode       = filter_SWORD_input(dfIn, dfNodeIn)

In [6]:
# select only included reaches for junction nodes
dfNodeConnection = get_connection_nodes(df[df['include_flag'] == '0'])

In [7]:
############################################
# Create downstream connection. Loop over all included reaches
############################################
wrongNormalReaches, wrongHOReaches = [], []
for i, r in df[df['include_flag'] == '0'].iterrows():
    IND = df[df.reach_id == r.reach_id].index[0]
    dn_reach, wrongNormal, wrongHO = get_connected_reach(r, 'dn', df, dfNode, dfNodeConnection)
    df.loc[IND, 'dn_connected_reach'] = dn_reach
    wrongNormalReaches.append(wrongNormal)
    wrongHOReaches.append(wrongHO)
df['dn_connected_reach'] = df['dn_connected_reach'].astype(float)


wrongNormalReaches =  dfNodeConnection[dfNodeConnection.Node_id.isin(wrongNormalReaches)].reach_id.values
wrongHOReaches = np.array(wrongHOReaches)
wrongHOReaches = wrongHOReaches[~np.isnan(wrongHOReaches)].astype(int)

In [8]:
############################################
# add river segment ID to dataframe
############################################
# create edges from down stream connections. 
        # Do not include reaches with no downstream connection, or with include_flag not 0
edges = df[['reach_id', 'dn_connected_reach']]
notIncludeReaches = df[df.include_flag != '0'].reach_id.values
edges = edges[(~edges['dn_connected_reach'].isna()) & 
                (~edges['reach_id'].isin(notIncludeReaches)) & 
                (~edges['dn_connected_reach'].isin(notIncludeReaches))
                ].values

# create network from networkx to create all separate river segments
G = nx.Graph()
G.add_edges_from(edges)

# create dataframe with river segments 
riverSegments   = list(nx.connected_components(G)) # all individual connected segments
dfriverSegments = pd.DataFrame(riverSegments)
dfriverSegments = dfriverSegments.unstack().reset_index(name='reach_id')[['level_1', 'reach_id']]
dfriverSegments = dfriverSegments.sort_values('level_1').dropna().rename(columns={"level_1": "river_segment"})
# merge river segments with main dataframe
df = df.merge(dfriverSegments, on = 'reach_id', how = 'left')
# give each row a river_segment value. For reaches without value create new values
df.loc[df['river_segment'].isna(), 'river_segment'] = np.arange(df[df['river_segment'].isna()].shape[0]) + df['river_segment'].max() + 1


In [9]:

############################################
# add river segment ID to dataframe
############################################
df, dfNode = split_river_segments(df, dfNode, min_reach_len_factor)

# find downstream river segment
df = find_dn_river_segment(df)


In [10]:
############################################
# remove lakeflags
############################################
def remove_lakeflag_ends(x):
    # Create a copy of the array for iteration
        arrIndices = np.arange(len(x))
        arrRemove = arrIndices.copy()
        for i in range(len(x)):
            if x[0] == 1:
                x = np.delete(x, 0)
                arrIndices = np.delete(arrIndices, 0)
            elif x[-1] == 1:
                x = np.delete(x, -1)
                arrIndices = np.delete(arrIndices, -1)
            else:
                # Stop the loop if condition is not satisfied
                break
        arrRemove = np.delete(arrRemove, arrIndices)
        return arrRemove

# group by combined reaches and take mean of lakeflag id and check if combined reach contains lakeflag with value 1
groupedLakeflag = df.groupby('combined_reach_id',as_index = False).agg(
                                mean_lake_flag=('lakeflag', 'mean'),
                                partial_lake =('lakeflag', lambda x: 1 in x.values))
# select all combined reaches with only lakeflag 1 reaches
lakes         = groupedLakeflag[(groupedLakeflag['mean_lake_flag'] % 1 == 0) & (groupedLakeflag['mean_lake_flag'] == 1)]['combined_reach_id'].values

# combined reaches partially existing of lakeflags
partial_lakes = groupedLakeflag[(groupedLakeflag['mean_lake_flag'] % 1 != 0) & (groupedLakeflag['partial_lake'] == True)]

# select lakes at the endpoint of a combined reach
removeReaches = []
for cr in partial_lakes['combined_reach_id'].values:
    pl = df[df['combined_reach_id'] == cr].sort_values('reach_order')

    removeLake = remove_lakeflag_ends(pl['lakeflag'].values)
    removeReaches.extend(pl.iloc[removeLake].reach_id.values)

# update include flag identified reaches
removeReaches.extend(lakes)
df = update_include_flag(df, removeReaches, '3')

# remove lakes and partial lakes from combined_reach_id and assign new combined
df.loc[df['include_flag'] == '3', 'combined_reach_id'] = np.arange(df[df['include_flag'] == '3'].shape[0]) + df['combined_reach_id'].max() + 1


In [11]:
############################################
# remove short reaches?
############################################
# remove reaches shorter than minimal requirement! 12*4*30
df['combined_reach_len']   = df.groupby('combined_reach_id')['reach_len'].transform('sum')
df['combined_reach_width'] = df.groupby('combined_reach_id')['width'].transform('mean')

dfNode['combined_reach_max_width'] = dfNode.groupby('combined_reach_id')['max_width'].transform('mean')
nmw = dfNode.groupby('combined_reach_id', as_index = False)['max_width'].mean()
nmw = nmw.rename(columns = {'max_width':'combined_reach_max_width'})
df = df.merge(nmw, on = 'combined_reach_id', how = 'left')

shortReaches = df.loc[df['combined_reach_len'] < min_reach_len_factor*30, 'reach_id'].values
df = update_include_flag(df, shortReaches, '4')


In [12]:
############################################
# remove missing Up, Down, dn_connected reaches
############################################
dfA = remove_updn_connection(df, True)


In [13]:
############################################
# change values for reach_order after removal of reaches
############################################
for cr in df.combined_reach_id.unique(): 
    rows = df[df.combined_reach_id == cr]
    rows = rows.sort_values('reach_order')
    df.loc[rows.index, 'reach_order'] = np.arange(rows.shape[0])


In [14]:

############################################
# add include_flag to dfNode
############################################
dfIncludeFlag = df[['reach_id', 'include_flag']].copy()
dfNode = dfNode.merge(dfIncludeFlag, on = 'reach_id', how = 'left')
    

In [20]:
df =  split_files(df, 5000)

In [28]:
df.shape[0]

15089

In [27]:
df.groupby(['path_segs', 'pfaf2'], as_index = False).size()

,path_segs,pfaf2,size
0,1,1,5
1,1,2,16
2,1,3,12
3,1,5,8
4,1,6,15
...,...,...,...
2675,978,6,7
2676,979,6,1
2677,980,6,1
2678,981,6,1


In [29]:
dfA = river_catchment_position(df)

Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected
Error no upstream end point detected


In [26]:
save = True
if save == True:
    dfSave = df.copy()                                         
    
    dfSave = split_files(dfSave, 5000)
    dfSave[['rch_id_up', 'rch_id_dn', 
            'rch_id_up_orig', 'rch_id_dn_orig']] = dfSave[['rch_id_up', 'rch_id_dn',
                                                           'rch_id_up_orig', 'rch_id_dn_orig']].astype(str)
               
    for group in dfSave['networkGroup'].unique():
        dfGroupSave = dfSave[dfSave['networkGroup'] == group]
        dfNodeSave = dfNode[dfNode.reach_id.isin(dfGroupSave.reach_id.values)]
        groupString = f'{group}'
        if group < 10:
            groupString =  f'0{groupString}'
        
        # dfGroupSave.to_file(directory + f'results/new_segments/vector/{fileName}_{groupString}_reach_new_segments.gpkg', driver = 'GPKG')
        # dfNodeSave.to_file(directory + f'results/new_segments/node/{fileName}_{groupString}_node_new_segments.gpkg', driver = 'GPKG')


In [48]:
dfSave.loc[dfSave['networkGroup'] == -1, ['reach_id', 'rch_id_up_orig','rch_id_dn_orig']].reach_id.values

array([], dtype=int64)

In [49]:
dfSave[dfSave['networkGroup'] == 2]




AttributeError: 'NoneType' object has no attribute 'copy'

,x,y,reach_id,reach_len,n_nodes,wse,wse_var,width,width_var,facc,...,river_segment,river_segment_order,reach_order,combined_reach_id,dn_river_segment,combined_reach_len,combined_reach_width,combined_reach_max_width,networkGraph,networkGroup
9789,129.901367,-2.955664,53200200315,10939.087229,55,2.000000,1.267081,42.00,3.145310e+02,224.814784,...,1089.0,2.0,0.0,6559.0,3264.0,10939.087229,42.00,45.900000,448,2
9790,129.907936,-3.022526,53200200325,11089.847781,55,12.600000,75.645584,54.00,6.770267e+02,190.685352,...,1089.0,1.0,0.0,6558.0,3264.0,11089.847781,54.00,56.563636,448,2
9791,129.892838,-3.097225,53200200335,10910.792315,55,63.200001,424.152750,54.00,5.530480e+02,156.251193,...,1089.0,0.0,0.0,6557.0,3264.0,10910.792315,54.00,61.663636,448,2
9792,129.857381,-2.912869,53200200165,8646.615720,43,1.300000,0.715589,42.00,8.511623e+02,181.107488,...,1090.0,4.0,0.0,6563.0,3266.0,8646.615720,42.00,43.639535,449,2
9793,129.846649,-2.953208,53200200175,8834.499047,44,4.500000,5.124918,42.00,3.253940e+02,161.879648,...,1090.0,3.0,0.0,6562.0,3266.0,8834.499047,42.00,47.840909,449,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15078,135.669162,-12.315542,56259100406,489.058000,1,0.000000,0.000000,152.00,5.003714e+03,266.034180,...,4654.0,0.0,0.0,12515.0,NaN,489.058000,152.00,NaN,544,2
15079,141.407867,-15.910474,56297000256,191.461710,1,0.000000,0.000000,368.50,2.624806e+03,4206.925137,...,4655.0,0.0,0.0,12516.0,NaN,191.461710,368.50,NaN,562,2
15080,142.405750,-28.694167,56800100546,146.586513,1,79.000000,0.000000,578.75,1.500625e+02,40724.000000,...,4656.0,0.0,0.0,12517.0,NaN,146.586513,578.75,NaN,627,2
15087,129.049184,-14.905278,56239000676,213.432509,1,0.200000,0.000000,1174.50,1.537854e+06,37.690655,...,4663.0,0.0,0.0,12524.0,NaN,213.432509,1174.50,NaN,519,2


In [21]:
dfInc = df[df['include_flag'] == '0']
dfInc[dfInc['combined_reach_max_width'].isna()]

AttributeError: 'NoneType' object has no attribute 'copy'

,x,y,reach_id,reach_len,n_nodes,wse,wse_var,width,width_var,facc,...,node_mean_max_width,dn_connected_reach,river_segment,river_segment_order,reach_order,combined_reach_id,dn_river_segment,combined_reach_len,combined_reach_width,combined_reach_max_width


# Test connect reaches

In [109]:
from connect_geometries import merge_centerlines
import matplotlib.pyplot as plt

In [101]:
df[df['reach_order'] == df['reach_order'].max()]


,x,y,reach_id,reach_len,n_nodes,wse,wse_var,width,width_var,facc,...,rch_id_up_orig,rch_id_dn_orig,n_rch_up_orig,n_rch_dn_orig,node_mean_max_width,dn_connected_reach,river_segment,river_segment_order,reach_order,combined_reach_id
6456,143.300405,-8.358088,53130000125,4050.197592,20,1.5,0.0,5188.0,0.0,15.771405,...,[53130000135],[53130000115],1,1,5188.0,NaN,711.0,23.0,14.0,4388.0


# test random

In [5]:
for c in ['oc', 'af', 'eu', 'na', 'as', 'sa']:
    files = glob(directory + f'results/new_segments/vector/{c}*.gpkg')
    for i, f in enumerate(files):
        dfTemp = gpd.read_file(f)
        if i == 0:
            df = dfTemp
        else:
            df = pd.concat([df, dfTemp])
    df.to_file(directory + f'results/new_segments/vector_cont/{c}_reaches.gpkg')

In [41]:
files = glob(directory + f'results/new_segments/vector/{'oc'}*.gpkg')
for i, f in enumerate(files):
        dfTemp = gpd.read_file(f)
        if i == 0:
            df = dfTemp
        else:
            df = pd.concat([df, dfTemp])

In [42]:
df.loc[df['reach_id'] == 53130101036, ['networkGraph', 'path_segs', 'pfaf2']]


,networkGraph,path_segs,pfaf2
1587,337,844,3


In [30]:
import ast

In [52]:
for N in df['networkGraph'].unique()[0:1]:    

    dfCatch = df[df['networkGraph'] == N]
    dfCatch = df[df['networkGraph'] == 337]

    # create edges
    edges = []
    for i, r in dfCatch.iterrows():
        DN = r['rch_id_dn_orig']
        if DN != 'nan':
        # if isinstance(DN, list):
            DN = ast.literal_eval(DN)
            for dn in DN:
                edges.append([r['reach_id'], dn])

    G = nx.DiGraph()
    G.add_edges_from(edges)
    RG = G.reverse()
    startPoints = [node for node, out_degree in RG.out_degree() if out_degree == 0]

    dfSegments = dfCatch.groupby(['path_segs', 'pfaf2'], as_index = False).size()


    for i, rs in dfSegments.iterrows():
        dfSegment = dfCatch[(dfCatch['path_segs'] == rs['path_segs']) & 
                            (dfCatch['pfaf2'] == rs['pfaf2'])]
        segmentIndex = dfSegment.index
        segmentReach = dfSegment.loc[dfSegment['dist_out'] == dfSegment['dist_out'].min(), 'reach_id'].iloc[0]
        
        upstreamReaches   = nx.ancestors(G, segmentReach)
        upstreamEndPoints = list(set(startPoints) & set(upstreamReaches))

        if len(upstreamEndPoints) > 0:
            rowID = dfCatch.loc[dfCatch['reach_id'].isin(upstreamEndPoints), 'dist_out'].idxmax()
            row   = dfCatch.loc[rowID]
            distVal   = row['dist_out']
            reachVal  = row['reach_id']

        else:
            print(f'new_reach_definition - river_catchment_position: Error no upstream end point detected ({segmentReach})', dfSegment.shape[0])
            reachVal = np.nan
            distVal = dfSegment['dist_out'].iloc[0]
        df.loc[segmentIndex, 'max_dist_out'] = distVal
        df.loc[segmentIndex, 'up_reach_id']  = reachVal
df['catchment_position'] = df['dist_out'] / df['max_dist_out']

##################################################################
# Single val network sections

In [51]:
dfSegment.loc[dfSegment['dist_out'] == dfSegment['dist_out'].min(), 'reach_id'].iloc[0]

53130101016

In [33]:
# files = glob(directory + 'input/SWOT_vector/as*.gpkg')[0]
# df = gpd.read_file(files)

for c in ['sa', 'as', 'na', 'eu', 'af', 'oc']:
    files = glob(directory + f'input/SWOT_vector/{c}*17*.gpkg')[0]
    print(files)
    d = gpd.read_file(files)
    print(d.shape[0])

/scratch/6256481/input/SWOT_vector/sa_sword_reaches_v17.gpkg
42133
/scratch/6256481/input/SWOT_vector/as_sword_reaches_v17.gpkg
100185
/scratch/6256481/input/SWOT_vector/na_sword_reaches_v17.gpkg
38696
/scratch/6256481/input/SWOT_vector/eu_sword_reaches_v17.gpkg
31103
/scratch/6256481/input/SWOT_vector/af_sword_reaches_v17.gpkg
21441
/scratch/6256481/input/SWOT_vector/oc_sword_reaches_v17.gpkg
15089


In [38]:
files = glob(directory + 'results/new_segments/vector/*.gpkg')
for f in files:
    d = gpd.read_file(f)
    dinc = d[d['include_flag'] == '0']
    print(d.shape[0], dinc['combined_reach_width'].min(), 
          dinc['combined_reach_max_width'].min())

5063 30.0 36.266666666666666
5003 25.0 27.370967741935484
5023 20.0 24.196428571428573
5018 20.0 24.88888888888889
5100 23.5 30.0
5051 22.5 27.16304347826087
5013 20.0 23.14
1259 32.597068786621094 38.04838709677419
6192 26.0 29.35576923076923
5514 34.5 34.6
5126 24.65745258331299 26.551367240793564
5111 22.06301498413086 36.5
7143 20.0 24.77777777777778
2017 36.0 37.02
6042 29.074541091918945 30.0
5028 30.0 30.0
5001 20.0 30.0
5910 30.027984619140625 31.336958445035496
5009 23.399999618530273 29.41993040197036
6316 21.4036283493042 23.95045304298401
5390 30.0 30.0
5286 28.5 29.257142857142856
5780 20.0 26.285714285714285
21074 20.5 24.916666666666668
4766 20.0 22.810344827586206
5227 20.5 24.265
5120 20.350515365600586 28.612166104997907
5484 22.386301040649414 27.125956654548645
6352 29.534046173095703 30.0
6635 21.0 21.63536434173584
7884 21.0 24.010416666666668
5304 22.0 26.223684210526315
5019 24.187387466430664 27.91304347826087
3877 20.0 22.3125
5004 20.924999237060547 26.988636

In [ ]:
glob(directory + 'input/SWOT_vector/{}*.gpkg')

['/scratch/6256481/input/SWOT_vector/eu_sword_reaches_v16.gpkg',
 '/scratch/6256481/input/SWOT_vector/af_sword_reaches_v17.gpkg',
 '/scratch/6256481/input/SWOT_vector/as_sword_reaches_v17.gpkg',
 '/scratch/6256481/input/SWOT_vector/eu_sword_reaches_v17.gpkg',
 '/scratch/6256481/input/SWOT_vector/na_sword_reaches_v17.gpkg',
 '/scratch/6256481/input/SWOT_vector/oc_sword_reaches_v17.gpkg',
 '/scratch/6256481/input/SWOT_vector/sa_sword_reaches_v17.gpkg']

In [94]:
nodeFiles = glob(directory + f'input/SWOT_nodes/{'as'}*v17*.gpkg')[0]
nodeFiles[-23:-5]

'as_sword_nodes_v17'

In [95]:
def split_input_file(cont, threshold):
    files     = glob(directory + f'input/SWOT_vector/{cont}*v17*.gpkg')[0]
    nodeFiles = glob(directory + f'input/SWOT_nodes/{cont}*v17*.gpkg')[0]
    filesName, nodeFilesName = files[-25:-5], nodeFiles[-23:-5]
    df     = gpd.read_file(files)
    dfNode = gpd.read_file(nodeFiles)

    edges = []
    for i, r in df.iterrows():
        rch = r[f'rch_id_dn']
        if len(rch) > 0:
            val = list(map(int, r[f'rch_id_dn'].split()))
            DN = val
            if isinstance(DN, list):
                for dn in DN:
                    edges.append([r['reach_id'], dn])

    G = nx.Graph()
    G.add_edges_from(edges)

    networks = list(nx.connected_components(G))
    dfN = pd.DataFrame(networks)
    dfN = dfN.unstack().reset_index(name='reach_id')[['level_1', 'reach_id']]
    dfN = dfN.sort_values('level_1').dropna().rename(columns={"level_1": "networkGraph"})

    dfNS = dfN.groupby('networkGraph', as_index = False).size()
    dfNS['cumSum'] = dfNS['size'].cumsum()


    dfNS.loc[dfNS['cumSum'] <= threshold, 'networkGroup']  = 0
    dfNS.loc[dfNS['cumSum'] > threshold, 'networkGroup']   = 1

    dfN = dfN.merge(dfNS[['networkGraph', 'networkGroup']], how = 'left', on ='networkGraph')
    df = df.merge(dfN, how = 'left', on = 'reach_id')

    for d in df['networkGroup'].unique():
        D  = df[df['networkGroup'] == d].copy()
        D  = D.drop(['networkGraph', 'networkGroup'], axis = 1)
        DN = dfNode[dfNode['reach_id'].isin(D['reach_id'].values)]
        
        D.to_file(directory + f'input_created/SWOT_vector_gpkg_split/{filesName}_{int(d)}.gpkg')
        DN.to_file(directory + f'input_created/SWOT_node_gpkg_split/{nodeFilesName}_{int(d)}.gpkg')
    return df
DF = split_input_file('as', 50000)

In [102]:

DF.shape[0], df.shape[0], DDF.shape[0], DDF.reach_id.nunique(),DDFN.reach_id.nunique() 

(100185, 100185, 100185, 100185, 100185)

In [101]:
F = glob(directory + 'input_created/SWOT_node_gpkg_split/*')
for i, f in enumerate(F):
    DDFNTemp = gpd.read_file(f)
    if i == 0:
        DDFN = DDFNTemp
    else:
        DDFN = pd.concat([DDFN, DDFNTemp])

In [60]:
df

,x,y,reach_id,reach_len,n_nodes,wse,wse_var,width,width_var,facc,...,path_order,path_segs,main_side,strm_order,end_reach,network,geometry,networkGraph,networkGroup_x,networkGroup_y
0,68.281779,68.165678,31101000021,2704.046410,14,0.000000,0.000000,626.0,2.032656e+04,3047.670948,...,882,836,0,2,0,24,"LINESTRING (68.30357 68.17384, 68.30285 68.173...",0,0.0,0.0
1,68.279176,68.148494,31101000033,1558.394545,8,0.000000,0.000000,310.0,1.009226e+04,3026.683343,...,882,836,0,2,0,24,"LINESTRING (68.27452 68.15493, 68.27453 68.154...",0,0.0,0.0
2,68.242202,68.109364,31101000041,13663.348149,68,2.000000,0.475677,192.0,6.792602e+03,3013.987137,...,882,836,0,2,0,24,"LINESTRING (68.28071 68.14183, 68.28000 68.141...",0,0.0,0.0
3,68.150080,68.074217,31101000051,12397.855688,62,2.100000,0.002186,127.0,1.906929e+03,2908.530898,...,882,836,0,2,0,24,"LINESTRING (68.21587 68.09514, 68.21516 68.094...",0,0.0,0.0
4,68.059351,68.015723,31101000061,13558.006122,68,2.500000,0.032183,108.0,8.799022e+02,2814.993545,...,882,836,0,2,0,24,"LINESTRING (68.11334 68.05539, 68.11392 68.055...",0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100180,72.951932,20.761194,45403903196,204.611387,1,4.000000,0.000000,75.0,4.338367e+02,1058.688145,...,3104,2707,0,2,2,34,"LINESTRING (72.95180 20.76052, 72.95185 20.760...",791,1.0,1.0
100181,67.503779,23.973542,45590001316,189.694959,1,0.000000,0.000000,704.0,9.393878e+02,49.745125,...,572,1168,2,-9999,2,5,"LINESTRING (67.50290 23.97326, 67.50319 23.973...",801,1.0,1.0
100182,68.073756,23.823601,45590001326,150.043592,1,1.000000,0.000000,6072.5,1.558735e+06,47.989112,...,3221,2875,2,-9999,2,5,"LINESTRING (68.07302 23.82360, 68.07331 23.823...",800,1.0,1.0
100183,68.280184,23.686241,45590001336,157.735261,1,0.000000,0.000000,366.0,2.018400e+02,23.057209,...,3283,2947,0,1,2,104,"LINESTRING (68.28066 23.68570, 68.28042 23.685...",802,1.0,1.0


In [56]:
dfNS

,networkGraph,size,cumSum,networkGroup
0,0,21,21,0.0
1,1,10,31,0.0
2,2,7,38,0.0
3,3,17,55,0.0
4,4,11,66,0.0
...,...,...,...,...
849,849,53,100144,1.0
850,850,12,100156,1.0
851,851,25,100181,1.0
852,852,2,100183,1.0
